### Armar un sistema de recomendación de películas. 

Éste consiste en recomendar películas a los usuarios basándose en películas similares, por lo que se debe encontrar la similitud de puntuación entre esa película y el resto de películas. 

Se ordenarán según el score de similaridad y devolverá:
- una lista de Python con 5 valores, 
- cada uno siendo el string del nombre de las películas con mayor puntaje 
- en orden descendente.

In [249]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

In [376]:
df = pd.read_csv('C:/Users/TheFercho/Desktop/SoyHenry/Labs/PI_TheFercho_labs/datasets/ML_dataset.csv')

In [377]:
df.columns

Index(['id', 'title', 'genres', 'original_language', 'spoken_languages',
       'popularity', 'production_companies', 'production_countries',
       'release_year', 'vote_count', 'vote_average'],
      dtype='object')

In [378]:
df.notna().sum()

id                      45344
title                   45344
genres                  45344
original_language       45344
spoken_languages        45344
popularity              45344
production_companies    45344
production_countries    45344
release_year            45344
vote_count              45344
vote_average            45344
dtype: int64

In [379]:
features = ['popularity', 'vote_average', 'vote_count', 'genres']
X = df[features]
missing_values = X.isnull().sum()
missing_values

popularity      0
vote_average    0
vote_count      0
genres          0
dtype: int64

In [380]:
df.loc[0,'title']

'Toy Story'

In [381]:
df[df['title'] == 'Batman']

,id,title,genres,original_language,spoken_languages,popularity,production_companies,production_countries,release_year,vote_count,vote_average
585,268,Batman,"Fantasy, Action",EN,"['English', 'Français']",19.106730,"['PolyGram Filmed Entertainment', 'The Guber-P...","['GB', 'US']",1989,2145,7.0
8597,2661,Batman,"Family, Adventure, Comedy, Science Fiction, Crime",EN,['English'],8.991532,"['Twentieth Century Fox Film Corporation', 'DC...",['US'],1966,209,6.1


In [336]:
#df[df.duplicated(subset='title', keep=False)]
df.drop_duplicates(subset='title', keep='first', inplace=True)


In [382]:
df.genres.unique()

array(['Animation, Comedy, Family', 'Adventure, Fantasy, Family',
       'Romance, Comedy', ..., 'Crime, Comedy, Action, Family',
       'Action, Mystery, Thriller, Horror',
       'Family, Animation, Romance, Comedy'], dtype=object)

In [383]:
def recomendacion_4(titulo):
            
    # Obtener el índice de la película que coincide con el título ingresado (case insensitive)
    indices = pd.Series(df.index, index=df['title'].str.lower()).drop_duplicates()
    titulo_lower = titulo.lower()
    
    if titulo_lower not in indices:
        return "Título no encontrado"
    
    idx = indices[titulo_lower][0]
    
    # Seleccionar las características relevantes para el algoritmo KNN
    features = ['popularity', 'vote_average', 'vote_count', 'genres']
    X = df[features]
    
    # Convertir las listas en columnas binarias utilizando MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    encoded_genres = pd.DataFrame(mlb.fit_transform(X['genres']), columns=mlb.classes_, index=X.index)
    
    # Convertir los nombres de características numéricas a cadenas
    numeric_features = X.drop('genres', axis=1)
    numeric_features.columns = numeric_features.columns.astype(str)
    
    # Aplicar PCA para reducir la dimensionalidad
    #El PCA es una técnica de reducción de dimensionalidad que 
    # transforma un conjunto de variables correlacionadas en un conjunto más pequeño de variables no correlacionadas 
    # llamadas componentes principales.
    pca = PCA(n_components=0.9)  # Mantenemos el 90% de la varianza explicada
    encoded_genres_pca = pca.fit_transform(encoded_genres)
    
    # Concatenar las características numéricas y las características transformadas por PCA
    X = pd.concat([numeric_features, pd.DataFrame(encoded_genres_pca)], axis=1)
    
    # Normalizar las características
    X = (X - X.mean()) / X.std()
    
    # Crear una instancia del algoritmo KNN
    knn = NearestNeighbors(n_neighbors=6)  # Consideramos 6 vecinos, incluyendo la película seleccionada
    
    # Resolvemos el siguiente error con la recomendacion de la propia libreria:  
    # Feature names are only supported if all input features have string names, but your input has ['int', 'str'] 
    # as feature name / column name types. If you want feature names to be stored and validated, 
    # you must convert them all to strings, by using: =>>>>> X.columns = X.columns.astype(str)
    X.columns = X.columns.astype(str)
    
    # Entrenar el modelo KNN
    knn.fit(X)
    
    # Obtener la distancia y los índices de los vecinos más cercanos
    distances, indices = knn.kneighbors(X.iloc[idx].values.reshape(1, -1))
    
    # Obtener los títulos de las películas más similares (excluyendo la película seleccionada)
    similar_movies = df['title'].iloc[indices[0][1:6]]
    
    return similar_movies

In [384]:
recomendacion_4('Batman')

c:\Users\TheFercho\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


1326                      Batman Returns
546       The Nightmare Before Christmas
12697                            Hancock
587      Snow White and the Seven Dwarfs
5828                  My Neighbor Totoro
Name: title, dtype: object

In [ ]:
def recomendacion_4(titulo):
            
    # Obtener el índice de la película que coincide con el título ingresado (case insensitive)
    indices = pd.Series(df.index, index=df['title'].str.lower()).drop_duplicates()
    return indices

In [366]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

def recomendacion(id_pelicula):
    # Cargar el dataframe
    
    # Eliminar los títulos duplicados en la columna 'title' (si no lo has hecho ya)
    df.drop_duplicates(subset='title', keep='first', inplace=True)
    
    # Verificar si hay valores faltantes en las columnas 'budget', 'revenue' y 'runtime'
    missing_values = df[['budget', 'revenue', 'runtime']].isnull().any()
    
    if missing_values.any():
        # Imputar los valores faltantes utilizando KNNImputer
        imputer = KNNImputer(n_neighbors=5)  # Puedes ajustar el número de vecinos según tu preferencia
        df[['budget', 'revenue', 'runtime']] = imputer.fit_transform(df[['budget', 'revenue', 'runtime']])
    
    # Obtener el índice de la película que coincide con el ID ingresado
    indices = pd.Series(df.index, index=df['id']).drop_duplicates()
    
    if id_pelicula not in indices:
        return "ID de película no encontrado"
    
    idx = indices[id_pelicula]
    
    # Seleccionar las características relevantes para el algoritmo KNN
    features = ['popularity', 'vote_average', 'vote_count', 'genres','budget', 'revenue', 'runtime']
    X = df[features]
    
    # Convertir las listas en columnas binarias utilizando MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    encoded_genres = pd.DataFrame(mlb.fit_transform(X['genres']), columns=mlb.classes_, index=X.index)
    
    # Convertir los nombres de características numéricas a cadenas
    numeric_features = X.drop('genres', axis=1)
    numeric_features.columns = numeric_features.columns.astype(str)
    
    # Aplicar PCA para reducir la dimensionalidad
    pca = PCA(n_components=0.9)  # Mantenemos el 90% de la varianza explicada
    encoded_genres_pca = pca.fit_transform(encoded_genres)
    
    # Concatenar las características numéricas y las características transformadas por PCA
    X = pd.concat([numeric_features, pd.DataFrame(encoded_genres_pca)], axis=1)
    
    # Normalizar las características
    X = (X - X.mean()) / X.std()
    
    # Crear una instancia del algoritmo KNN
    knn = NearestNeighbors(n_neighbors=6)  # Consideramos 6 vecinos, incluyendo la película seleccionada
    
    # Resolvemos el siguiente error con la recomendacion de la propia libreria:  
    # Feature names are only supported if all input features have string names, but your input has ['int', 'str'] 
    # as feature name / column name types. If you want feature names to be stored and validated, 
    # you must convert them all to strings, by using: =>>>>> X.columns = X.columns.astype(str)
    X.columns = X.columns.astype(str)
    
    # Entrenar el modelo KNN
    knn.fit(X)
    
    # Obtener la
    similar_movies = df['title'].iloc[indices[0][1:6]]
    
    return similar_movies

In [367]:
recomendacion('Batman')

KeyError: "None of [Index(['budget', 'revenue', 'runtime'], dtype='object')] are in the [columns]"